1. Create a table called employees with the following structure?
: emp_id (integer, should not be NULL and should be a primary key)Q
: emp_name (text, should not be NULL)Q
: age (integer, should have a check constraint to ensure the age is at least 18)Q
: email (text, should be unique for each employee)Q
: salary (decimal, with a default value of 30,000).

Write the SQL query to create the above table with all constraints.

In [ ]:
import sqlite3
import pandas as pd

# Create an in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [ ]:
cursor.execute('''
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY NOT NULL,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL DEFAULT 30000
);
''')

# Insert a sample record
cursor.execute('''
INSERT INTO employees (emp_id, emp_name, age, email)
VALUES (1, 'John Doe', 28, 'john@example.com');
''')

# View the data
pd.read_sql_query("SELECT * FROM employees", conn)


2. Explain the purpose of constraints and how they help maintain data integrity in a database. Provide
examples of common types of constraints.

-> Constraints enforce rules on table columns to protect the integrity of data.

Common constraints:

PRIMARY KEY: Uniquely identifies each row

FOREIGN KEY: Maintains referential integrity

UNIQUE: Prevents duplicate values

NOT NULL: Ensures no missing data

CHECK: Validates column values

DEFAULT: Sets a default value if none is provided

3.Why would you apply the NOT NULL constraint to a column? Can a primary key contain NULL values? Justify
your answer.

-> Use NOT NULL to ensure that a column always has a value.

A PRIMARY KEY cannot be NULL because it must uniquely identify each record.

4. Explain the steps and SQL commands used to add or remove constraints on an existing table. Provide an
example for both adding and removing a constrain.

In [ ]:
cursor.execute('DROP TABLE IF EXISTS demo_table')
cursor.execute('''
CREATE TABLE demo_table (
    id INTEGER,
    age INTEGER CHECK (age >= 18)
);
''')


5. Explain the consequences of attempting to insert, update, or delete data in a way that violates constraints.
Provide an example of an error message that might occur when violating a constraint.

In [ ]:
try:
    cursor.execute('''
    INSERT INTO employees (emp_id, emp_name, age, email)
    VALUES (2, 'Jane Smith', 16, 'jane@example.com'); -- age < 18 violates CHECK
    ''')
except Exception as e:
    print("Error:", e)


6. You created a products table without constraints as follows:

CREATE TABLE products (

    product_id INT,

    product_name VARCHAR(50),

    price DECIMAL(10, 2));
Now, you realise that?
: The product_id should be a primary keyQ
: The price should have a default value of 50.00

In [ ]:
# Drop if exists
cursor.execute("DROP TABLE IF EXISTS products")

# Recreate with constraints
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price DECIMAL(10, 2) DEFAULT 50.00
);
''')

# Insert a sample product without price to test default
cursor.execute("INSERT INTO products (product_id, product_name) VALUES (1, 'Laptop')")

# View data
pd.read_sql_query("SELECT * FROM products", conn)


7. Write a query to fetch the student_name and class_name for each student using an INNER JOIN

In [ ]:
# Create Students table
cursor.execute('''
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    student_name TEXT,
    class_id INTEGER
);
''')

# Create Classes table
cursor.execute('''
CREATE TABLE classes (
    class_id INTEGER PRIMARY KEY,
    class_name TEXT
);
''')

# Insert sample students
cursor.executemany('''
INSERT INTO students (student_id, student_name, class_id)
VALUES (?, ?, ?)
''', [
    (1, 'Alice', 101),
    (2, 'Bob', 102),
    (3, 'Charlie', 101)
])

# Insert sample classes
cursor.executemany('''
INSERT INTO classes (class_id, class_name)
VALUES (?, ?)
''', [
    (101, 'Math'),
    (102, 'Science'),
    (103, 'History')
])

# INNER JOIN query to get student_name and class_name
pd.read_sql_query('''
SELECT s.student_name, c.class_name
FROM students s
INNER JOIN classes c ON s.class_id = c.class_id;
''', conn)


8. Write a query that shows all order_id, customer_name, and product_name, ensuring that all products are
listed even if they are not associated with an order

Hint: (use INNER JOIN and LEFT JOIN)5

In [ ]:
import sqlite3
import pandas as pd

# Create an in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [ ]:
# Insert into Orders
cursor.executemany('''
INSERT INTO Orders (order_id, order_date, customer_id)
VALUES (?, ?, ?)
''', [
    (1, '2024-01-01', 101),
    (2, '2024-01-03', 102)
])

# Insert into Customers
cursor.executemany('''
INSERT INTO Customers (customer_id, customer_name)
VALUES (?, ?)
''', [
    (101, 'Alice'),
    (102, 'Bob')
])

# Insert into Products
cursor.executemany('''
INSERT INTO Products (product_id, product_name, order_id)
VALUES (?, ?, ?)
''', [
    (1, 'Laptop', 1),
    (2, 'Phone', None)
])


In [ ]:
pd.read_sql_query('''
SELECT p.order_id, c.customer_name, p.product_name
FROM Products p
LEFT JOIN Orders o ON p.order_id = o.order_id
LEFT JOIN Customers c ON o.customer_id = c.customer_id;
''', conn)


9. Write a query to find the total sales amount for each product using an INNER JOIN and the SUM() function.

In [ ]:
# Insert into Sales
cursor.executemany('''
INSERT INTO Sales (sale_id, product_id, amount)
VALUES (?, ?, ?)
''', [
    (1, 101, 500),
    (2, 102, 300),
    (3, 101, 700)
])

# Insert into ProductList
cursor.executemany('''
INSERT INTO ProductList (product_id, product_name)
VALUES (?, ?)
''', [
    (101, 'Laptop'),
    (102, 'Phone')
])


In [ ]:
pd.read_sql_query('''
SELECT p.product_name, SUM(s.amount) AS total_sales
FROM Sales s
JOIN ProductList p ON s.product_id = p.product_id
GROUP BY p.product_name;
''', conn)


10. Write a query to display the order_id, customer_name, and the quantity of products ordered by each
customer using an INNER JOIN between all three tables

In [ ]:
# Create Orders2 table
cursor.execute('''
CREATE TABLE Orders2 (
    order_id INTEGER PRIMARY KEY,
    order_date TEXT,
    customer_id INTEGER
);
''')

# Create Customers2 table
cursor.execute('''
CREATE TABLE Customers2 (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT
);
''')

# Create Order_Details table
cursor.execute('''
CREATE TABLE Order_Details (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER
);
''')

# Insert into Orders2
cursor.executemany('''
INSERT INTO Orders2 (order_id, order_date, customer_id)
VALUES (?, ?, ?)
''', [
    (1, '2024-01-02', 1),
    (2, '2024-01-05', 2)
])

# Insert into Customers2
cursor.executemany('''
INSERT INTO Customers2 (customer_id, customer_name)
VALUES (?, ?)
''', [
    (1, 'Alice'),
    (2, 'Bob')
])

# Insert into Order_Details
cursor.executemany('''
INSERT INTO Order_Details (order_id, product_id, quantity)
VALUES (?, ?, ?)
''', [
    (1, 101, 2),
    (1, 102, 1),
    (2, 101, 3)
])

In [ ]:
pd.read_sql_query('''
SELECT
    o.order_id,
    c.customer_name,
    od.quantity
FROM
    Orders2 o
JOIN
    Customers2 c ON o.customer_id = c.customer_id
JOIN
    Order_Details od ON o.order_id = od.order_id;
''', conn)

**SQL Commands**

1-Identify the primary keys and foreign keys in maven movies db. Discuss the differences.

In [ ]:
import sqlite3
import pandas as pd

# Connect to MavenMovies SQLite DB (you must upload the database file)
conn = sqlite3.connect("mavenmovies.db")  # Upload mavenmovies.db to your Colab environment


Primary Keys:
- actor.actor_id
- customer.customer_id
- film.film_id
- inventory.inventory_id
- rental.rental_id
- payment.payment_id

Foreign Keys:
- film_actor.actor_id - actor.actor_id
- film_actor.film_id - film.film_id
- inventory.film_id - film.film_id
- inventory.store_id - store.store_id
- rental.inventory_id - inventory.inventory_id
- rental.customer_id - customer.customer_id
- payment.customer_id - customer.customer_id
- payment.staff_id - staff.staff_id

2- List all details of actors.

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your mavenmovies.db file here


Saving Mavenmovies.sql to Mavenmovies (4).sql


In [ ]:
import os
os.listdir()


['.config',
 'mavenmovies.db',
 'Mavenmovies (1).sql',
 'Mavenmovies.sql',
 'Mavenmovies (3).sql',
 'drive',
 'Mavenmovies (2).sql',
 'sql database.pdf',
 'Mavenmovies (4).sql',
 'sample_data']

In [ ]:
['mavenmovies.db']


['mavenmovies.db']

3 -List all customer information from DB.

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Mavenmovies (2).sql to Mavenmovies (2) (1).sql


In [66]:
import sqlite3
import pandas as pd

# If uploaded file is named something different, use that exact name
conn = sqlite3.connect('mavenmovies.db') # Changed from mavenmovies.sql to mavenmovies.db

In [ ]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name


4 -List different countries.

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Mavenmovies.sql to Mavenmovies (1).sql


In [ ]:
import sqlite3
import pandas as pd

# Connect to the uploaded database file
conn = sqlite3.connect('mavenmovies.db')


In [ ]:
# Check if 'country' table is present in your DB
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

5 -Display all active customers.



In [ ]:
import sqlite3
import pandas as pd

# Connect to the uploaded database file
conn = sqlite3.connect("mavenmovies.db")


In [ ]:
import sqlite3
import pandas as pd


In [ ]:
conn = sqlite3.connect("mavenmovies.db")


In [ ]:
# Check if 'customer' table is present in your DB or list available tables
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name


6 -List of all rental IDs for customer with ID 1.

In [ ]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name


Question 1:

Retrieve the total number of rentals made in the Sakila database.

Hint: Use the COUNT() function.

In [65]:
query1 = """
SELECT
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    SUM(p.amount) AS total_spent
FROM
    customer c
JOIN
    payment p ON c.customer_id = p.customer_id
GROUP BY
    c.customer_id
ORDER BY
    total_spent DESC;
"""
pd.read_sql_query(query1, conn)


DatabaseError: Execution failed on sql '
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    SUM(p.amount) AS total_spent
FROM 
    customer c
JOIN 
    payment p ON c.customer_id = p.customer_id
GROUP BY 
    c.customer_id
ORDER BY 
    total_spent DESC;
': no such table: customer